In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
cifartrain=datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

cifartest=datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_dataloader=DataLoader(dataset=cifartrain,batch_size=64,shuffle=True)
test_dataloader=DataLoader(dataset=cifartest,batch_size=64,shuffle=True)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = nn.Softmax(dim=1)(model(X.cuda()))
        loss = loss_fn(pred, y.cuda())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X.cuda())
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = nn.Softmax(dim=1)(model(X.cuda()))
            test_loss += loss_fn(pred, y.cuda()).item()
            correct += (pred.argmax(1) == y.cuda()).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [8]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size = 64
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302252  [    0/50000]
loss: 2.302007  [ 6400/50000]
loss: 2.301076  [12800/50000]
loss: 2.302584  [19200/50000]
loss: 2.298417  [25600/50000]
loss: 2.301682  [32000/50000]
loss: 2.298552  [38400/50000]
loss: 2.295808  [44800/50000]
Test Error: 
 Accuracy: 10.6%, Avg loss: 2.297053 

Epoch 2
-------------------------------
loss: 2.295161  [    0/50000]
loss: 2.297308  [ 6400/50000]
loss: 2.301603  [12800/50000]
loss: 2.291112  [19200/50000]
loss: 2.295915  [25600/50000]
loss: 2.296938  [32000/50000]
loss: 2.289425  [38400/50000]
loss: 2.285409  [44800/50000]
Test Error: 
 Accuracy: 12.4%, Avg loss: 2.285805 

Epoch 3
-------------------------------
loss: 2.291604  [    0/50000]
loss: 2.295363  [ 6400/50000]
loss: 2.281157  [12800/50000]
loss: 2.299959  [19200/50000]
loss: 2.273010  [25600/50000]
loss: 2.245944  [32000/50000]
loss: 2.265041  [38400/50000]
loss: 2.253691  [44800/50000]
Test Error: 
 Accuracy: 16.4%, Avg loss: 2.267403 

Epoc